In [187]:
import pandas as pd
import seaborn as sns

In [188]:
df_madrid = pd.read_csv("../data/madrid-coordinates.csv")

In [189]:
df_madrid.columns

Index(['id', 'title', 'subtitle', 'sq_mt_built', 'sq_mt_useful', 'n_rooms',
       'n_bathrooms', 'n_floors', 'sq_mt_allotment', 'latitude', 'longitude',
       'raw_address', 'is_exact_address_hidden', 'street_name',
       'street_number', 'portal', 'floor', 'is_floor_under', 'door',
       'neighborhood_id', 'operation', 'rent_price', 'rent_price_by_area',
       'is_rent_price_known', 'buy_price', 'buy_price_by_area',
       'is_buy_price_known', 'house_type_id', 'is_renewal_needed',
       'is_new_development', 'built_year', 'has_central_heating',
       'has_individual_heating', 'are_pets_allowed', 'has_ac',
       'has_fitted_wardrobes', 'has_lift', 'is_exterior', 'has_garden',
       'has_pool', 'has_terrace', 'has_balcony', 'has_storage_room',
       'is_furnished', 'is_kitchen_equipped', 'is_accessible',
       'has_green_zones', 'energy_certificate', 'has_parking',
       'has_private_parking', 'has_public_parking',
       'is_parking_included_in_price', 'parking_price', 'is

#### Columns to use:
- sq_mt_built
- sq_mt_useful
- n_rooms
- n_bathrooms
- latitude
- longitude
- raw_address
- neighborhood_id
- buy_price
- house_type_id
- is_new_development
- is_renewal_needed
- has_lift
- is_exterior
- has_parking
- floor

In [190]:
columns = ["id",
            "sq_mt_built",
          "sq_mt_useful",
          "n_rooms",
          "n_bathrooms",
           "latitude",
           "longitude",
           "raw_address",
           "neighborhood_id",
           "buy_price",
           "house_type_id",
           "is_new_development",
           "is_renewal_needed",
           "has_lift",
           "is_exterior",
           "has_parking",
           "floor"
          ]

In [191]:
new_madrid = df_madrid[columns]

In [170]:
new_madrid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16277 entries, 0 to 16276
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  16277 non-null  int64  
 1   sq_mt_built         16216 non-null  float64
 2   sq_mt_useful        6753 non-null   float64
 3   n_rooms             16277 non-null  int64  
 4   n_bathrooms         16264 non-null  float64
 5   latitude            14266 non-null  float64
 6   longitude           14266 non-null  float64
 7   raw_address         16277 non-null  object 
 8   neighborhood_id     16277 non-null  object 
 9   buy_price           16277 non-null  int64  
 10  house_type_id       15946 non-null  object 
 11  is_new_development  15514 non-null  object 
 12  is_renewal_needed   16277 non-null  bool   
 13  has_lift            14955 non-null  object 
 14  is_exterior         14465 non-null  object 
 15  has_parking         16277 non-null  bool   
 16  floo

#### Let's filter by latitude and longitude:

In [192]:
new_madrid = new_madrid[(new_madrid.latitude > 40.34) & 
                      (new_madrid.latitude < 40.51) & 
                      (new_madrid.longitude > -3.8) & 
                      (new_madrid.longitude < -3.6)]

#### We can estimate "useful square meters" from the other apartments:

### Cleaning "sq_mt_built" and "sq_mt_useful":

In [193]:
new_madrid["sq_mt_coef"] = new_madrid["sq_mt_useful"]/new_madrid["sq_mt_built"]

In [194]:
sq_mt_ratio = new_madrid["sq_mt_coef"].mean()

In [195]:
sq_mt_ratio

0.856862301863558

In [196]:
new_madrid["sq_mt_useful"].fillna(round(new_madrid["sq_mt_built"] * sq_mt_ratio, 2), inplace=True)

In [197]:
new_madrid["sq_mt_built"].fillna(round(new_madrid["sq_mt_useful"] / sq_mt_ratio, 2), inplace=True)

### Cleaning "has_garden", "has_balcony", "has_pool", "has_terrace", "has_storage_room":

In [198]:
def clean_cols_has(df, column):
    df[column].fillna(0, inplace=True)
    df.loc[(df[column] == True), column] = 1
    df[column] = df[column].astype(int)
    return df

In [199]:
new_madrid.reset_index(drop = True)

,id,sq_mt_built,sq_mt_useful,n_rooms,n_bathrooms,latitude,longitude,raw_address,neighborhood_id,buy_price,house_type_id,is_new_development,is_renewal_needed,has_lift,is_exterior,has_parking,floor,sq_mt_coef
0,21741,70.0,59.98,3,1.0,40.359340,-3.698374,Calle de la del Manojo de Rosas,Neighborhood 132: Los Ángeles (1796.68 €/m2) -...,129900,HouseType 1: Pisos,False,True,True,True,False,4,NaN
1,21740,94.0,54.00,2,2.0,40.344630,-3.715191,"Calle del Talco, 68",Neighborhood 134: San Andrés (1617.18 €/m2) - ...,144247,HouseType 1: Pisos,False,False,True,True,False,1,0.574468
2,21738,108.0,90.00,2,2.0,40.357722,-3.685029,Carretera de Villaverde a Vallecas,Neighborhood 133: Los Rosales (1827.79 €/m2) -...,260000,HouseType 1: Pisos,False,False,True,True,True,4,0.833333
3,21737,126.0,114.00,4,2.0,40.343389,-3.712416,geologia,Neighborhood 134: San Andrés (1617.18 €/m2) - ...,195000,HouseType 1: Pisos,False,False,False,True,True,3,0.904762
4,21736,120.0,100.00,5,2.0,40.348149,-3.706239,Avenida Real de Pinto,Neighborhood 134: San Andrés (1617.18 €/m2) - ...,190000,HouseType 1: Pisos,False,False,False,True,True,1,0.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10877,32,228.0,195.36,4,2.0,40.389037,-3.692823,Calle Teresa López Valcárcel,Neighborhood 4: Legazpi (4486.93 €/m2) - Distr...,895000,HouseType 1: Pisos,False,False,True,True,True,5,NaN
10878,20,74.0,63.41,2,2.0,40.423552,-3.796780,Calle Timón,Neighborhood 11: Timón (2879.88 €/m2) - Distri...,325000,HouseType 1: Pisos,False,False,True,NaN,True,2,NaN
10879,11,276.0,236.49,4,4.0,40.408853,-3.791879,Calle Siroco,Neighborhood 11: Timón (2879.88 €/m2) - Distri...,690000,HouseType 2: Casa o chalet,False,False,NaN,NaN,True,NaN,NaN
10880,9,267.0,215.00,5,3.0,40.450461,-3.608672,Avenida de Logroño,Neighborhood 9: Campo de las Naciones-Corralej...,790000,HouseType 2: Casa o chalet,NaN,False,NaN,NaN,True,NaN,0.805243


#### House Type:

In [200]:
new_madrid.house_type_id.value_counts()

HouseType 1: Pisos            9401
HouseType 5: Áticos            481
HouseType 2: Casa o chalet     442
HouseType 4: Dúplex            315
Name: house_type_id, dtype: int64

In [201]:
house_types = {
    'HouseType 1: Pisos': 0,
    'HouseType 2: Casa o chalet': 3,
    'HouseType 5: Áticos': 2,
    'HouseType 4: Dúplex': 1
}

In [202]:
new_madrid["house_type"] = new_madrid.house_type_id.map(house_types)

In [203]:
new_madrid.house_type.value_counts()

0.0    9401
2.0     481
3.0     442
1.0     315
Name: house_type, dtype: int64

### Let's keep cleaning columns:

In [204]:
def boolean_to_binary(df, column):
    df.loc[(df[column] == True), column] = 1
    df.loc[(df[column] == False), column] = 0
    df[column] = df[column].astype(int)
    return df

#### Drop "sq_mt_coef" column:

In [205]:
new_madrid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10882 entries, 1 to 16276
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  10882 non-null  int64  
 1   sq_mt_built         10876 non-null  float64
 2   sq_mt_useful        10876 non-null  float64
 3   n_rooms             10882 non-null  int64  
 4   n_bathrooms         10872 non-null  float64
 5   latitude            10882 non-null  float64
 6   longitude           10882 non-null  float64
 7   raw_address         10882 non-null  object 
 8   neighborhood_id     10882 non-null  object 
 9   buy_price           10882 non-null  int64  
 10  house_type_id       10639 non-null  object 
 11  is_new_development  10334 non-null  object 
 12  is_renewal_needed   10882 non-null  bool   
 13  has_lift            10197 non-null  object 
 14  is_exterior         9863 non-null   object 
 15  has_parking         10882 non-null  bool   
 16  floo

In [185]:
new_madrid.drop("sq_mt_coef", axis=1, inplace=True)

In [211]:
new_madrid.is_new_development.value_counts()

False    9858
True      476
Name: is_new_development, dtype: int64

#### Drop nulls:

In [186]:
new_madrid.isna().sum()

id                       0
sq_mt_built              6
sq_mt_useful             6
n_rooms                  0
n_bathrooms             10
latitude                 0
longitude                0
raw_address              0
neighborhood_id          0
buy_price                0
house_type_id          243
is_new_development     548
is_renewal_needed        0
has_lift               685
is_exterior           1019
has_parking              0
floor                  725
house_type             243
dtype: int64

In [140]:
madrid = new_madrid.dropna(axis=0, how="any")

### Let's prepare some last columns:

#### Has_lift column:

In [142]:
madrid = boolean_to_binary(madrid, "has_lift")

/home/ccastrob/anaconda3/envs/ironhack/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-136-89f2cc17128b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].astype(int)


#### Has_parking column:

In [143]:
madrid.has_parking.value_counts()

False    6475
True     2655
Name: has_parking, dtype: int64

In [144]:
madrid = boolean_to_binary(madrid, "has_parking")

/home/ccastrob/anaconda3/envs/ironhack/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
<ipython-input-136-89f2cc17128b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].astype(int)


#### n_bathrooms column:

In [145]:
madrid["n_bathrooms"] = madrid["n_bathrooms"].astype(int)

<ipython-input-145-2f0d247d1d86>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  madrid["n_bathrooms"] = madrid["n_bathrooms"].astype(int)


#### Floor column:

In [146]:
madrid.floor.value_counts()

1                       2104
2                       1676
3                       1417
4                       1159
Bajo                     967
5                        625
6                        423
7                        280
8                        164
Entreplanta exterior     144
9                         81
Semi-sótano exterior      32
Entreplanta interior      21
Sótano interior           18
Semi-sótano interior      17
Sótano exterior            2
Name: floor, dtype: int64

In [147]:
floors = {
    "Bajo" : 0,
    "Entreplanta" : 0.5,
    "Entreplanta exterior" : 0.5,
    "Entreplanta interior" : 0.5,
    "Semi-sótano" : -0.5,
    "Semi-sótano exterior" : -0.5,
    "Semi-sótano interior" : -0.5,
    "Sótano" : -1,
    "Sótano interior" : -1,
    "Sótano exterior" : -1,    
}

In [148]:
madrid = madrid.replace({"floor": floors})

In [149]:
madrid["floor"] = madrid["floor"].astype(float)

#### Let's change type of "is_exterior", "is_renewal_needed" and "is_new_development":

In [150]:
madrid = boolean_to_binary(madrid, "is_renewal_needed")

In [151]:
madrid = boolean_to_binary(madrid, "is_exterior")

In [152]:
madrid = boolean_to_binary(madrid, "is_new_development")

#### Let's drop house_type_id column because we already has a house_type column:

In [153]:
madrid.drop("house_type_id", axis=1, inplace=True)

#### Final cleaning: neigborhood_id column:

In [155]:
madrid["value_m2"] = madrid["neighborhood_id"].str.split("(", n=1, expand=True)[1].str.split(" ", n=1, expand=True)[0]

In [156]:
madrid["neighborhood"] = madrid["neighborhood_id"].str.split(":", n=1, expand=True)[1].str.split("(", n=1, expand=True)[0].str.strip()

In [157]:
madrid["district"] = madrid["neighborhood_id"].str.split(":", n=2, expand=True)[2].str.strip()

In [158]:
madrid.loc[(madrid["value_m2"] == "None"), "value_m2"] = 0

In [159]:
madrid.value_m2 = madrid.value_m2.astype(float)

#### I will modify type of house_type to int:

In [160]:
madrid.house_type = madrid.house_type.astype(int)

#### I will check the final dataframe clean:

In [161]:
madrid.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9130 entries, 1 to 16250
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  9130 non-null   int64  
 1   sq_mt_built         9130 non-null   float64
 2   sq_mt_useful        9130 non-null   float64
 3   n_rooms             9130 non-null   int64  
 4   n_bathrooms         9130 non-null   int64  
 5   latitude            9130 non-null   float64
 6   longitude           9130 non-null   float64
 7   raw_address         9130 non-null   object 
 8   neighborhood_id     9130 non-null   object 
 9   buy_price           9130 non-null   int64  
 10  is_new_development  9130 non-null   int64  
 11  is_renewal_needed   9130 non-null   int64  
 12  has_lift            9130 non-null   int64  
 13  is_exterior         9130 non-null   int64  
 14  has_parking         9130 non-null   int64  
 15  floor               9130 non-null   float64
 16  house

#### Let's export the final result:

In [164]:
madrid.to_csv("../data/clean-madrid-v2.csv", index = False)